In [1]:
# import packages
from ipyleaflet import Map, basemaps, basemap_to_tiles, DrawControl, TileLayer, LayersControl
import json
import panel as pn
pn.extension("ipywidgets", sizing_mode="stretch_width")
import requests
import ipywidgets as widgets

# Visualizing GeoTiffs in MAAP

In [64]:
# Declare variables
provider = 'NASA_MAAP'
cmr_search_url = "https://cmr.maap-project.org/search"
collections_search_url = f"{cmr_search_url}/collections.json?provider={provider}&page_size=100"
granules_search_url = f"{cmr_search_url}/collections.json"

In [3]:
# Make a selection of collections
response = requests.get(collections_search_url)

<Response [200]>

In [4]:
collections = json.loads(response.text)['feed']['entry']

In [5]:
collection_names = list(collection['short_name'] for collection in collections)

In [162]:
# create a drop down of collections

collection_selection = widgets.Dropdown(
    options=collection_names,
    value='ABLVIS2',
    description='',
    disabled=False,
    layout=widgets.Layout(flex='1 1 auto', width='auto')
)

In [188]:
from IPython.display import display, clear_output

bbox = {
    'lower longitude': 0,
    'lower latitude': 0,
    'upper longitude': 0,
    'upper latitude': 0    
}
bbox_widgets = {}

for k, v in bbox.items():
    bbox_widgets[k] = widgets.FloatText(
        value=bbox[k],
        description=k,
        disabled=False,
        layout=widgets.Layout(flex='1 1 auto', width='auto')
    )
    
bbox_twobytwo = widgets.TwoByTwoLayout(
    top_left=bbox_widgets['upper longitude'],
    top_right=bbox_widgets['upper latitude'],
    bottom_left=bbox_widgets['lower longitude'],
    bottom_right=bbox_widgets['lower latitude'] 
)

## Search for granules

In [198]:
granule_url_selection = widgets.Dropdown(
    options=['Granule URLs will appear here following search.'],
    value='Granule URLs will appear here following search.',
    description='',
    disabled=False,
    layout=widgets.Layout(flex='1 1 auto', width='auto')
)

messages = widgets.Output()

def tifs_only(granule_objects):
    tifs = []
    for granule in granule_objects:
        for link in granule['links']:
            if link['href'].endswith('.tif'):
                tifs.append(link['href'])
    return tifs

def search_for_granules(b):
    bbox = [i.value for i in bbox_widgets.values()]
    bbox_string = ','.join(map(str, bbox))
    search_url_with_params = f"{cmr_search_url}/granules.json?short_name={collection_selection.value}&bounding_box[]={bbox_string}"
    granules_response = requests.get(search_url_with_params)
    granule_objects = json.loads(granules_response.text)['feed']['entry']
    if len(granule_objects) == 0:
        with messages:
            print('No granules found.')
    else:
        tif_urls = tifs_only(granule_objects)
        if len(tif_urls) > 0:
            granule_url_selection.options = tif_urls
        else:
            with messages:
                messages.clear_output()
                print('There are no geotiffs in this collection.')

granules_button = widgets.Button(description="Run search")
granules_button.on_click(search_for_granules)

In [199]:
collection_title = widgets.HTML('<h3>Select a collection: </h3>')
bbox_title = widgets.HTML('<h3>(Optional) Draw a bounding box on the map to search for granules. </h3>')
search_granules_title = widgets.HTML('<h3>Run Search</h3><h4>URLs for GeoTiffs discovered from search will be listed in the dropdown below </h4>')
search_box = widgets.Box(children=[search_granules_title, granules_button])
widgets.VBox([
    collection_title,
    collection_selection,
    bbox_title,
    bbox_twobytwo,
    search_granules_title,
    granules_button,
    granule_url_selection,
    messages
])

In [185]:
m = Map(
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    center=(0, 0),
    zoom=4
)
draw_control = DrawControl()

draw_control.rectangle = {
    "shapeOptions": {
        "fillColor": "#fca45d",
        "color": "#fca45d",
        "fillOpacity": 0.2
    }
}
m.add_control(draw_control)

def handle_draw(self, action, geo_json):
    """Do something with the GeoJSON when it's drawn on the map""" 
    bbox = geo_json['geometry']['coordinates'][0]
    bbox_widgets['lower longitude'].value = bbox[0][0]
    bbox_widgets['lower latitude'].value = bbox[0][1]
    bbox_widgets['upper longitude'].value = bbox[2][0]    
    bbox_widgets['upper latitude'].value = bbox[1][1]

draw_control.on_draw(handle_draw)
m.add_control(LayersControl())
m

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

In [62]:
def update_map(b):
    granule_s3_url = granule_url_selection.value
    if collection_selection.value == 'GEDI_L4B_Gridded_Biomass_2017':
        tititler_url = "https://l40l1vwmvi.execute-api.us-west-2.amazonaws.com"
    else:
        tititler_url = "https://titiler.maap-project.org"
    rescale = "0,400"
    colormap_name = "gist_earth_r"
    color_formula = "gamma r 2"
    tiles_url = tititler_url + "/cog/tiles/{z}/{x}/{y}.png?url=" + f"{granule_s3_url}&rescale={rescale}&colormap_name={colormap_name}&color_formula={color_formula}"
    tile_layer=TileLayer(url=tiles_url)
    m.add_layer(tile_layer)
    return m

## Update the map

In [63]:
update_map_button = widgets.Button(description="Update Map")
update_map_button.on_click(update_map)
display(update_map_button)

Button(description='Update Map', style=ButtonStyle())

https://l40l1vwmvi.execute-api.us-west-2.amazonaws.com/cog/tiles/{z}/{x}/{y}.png?url=s3://ornl-cumulus-prod-protected/gedi/GEDI_L4B_Gridded_Biomass/data/GEDI04_B_MW019MW138_02_002_05_R01000M_PS.tif&rescale=0,400&colormap_name=gist_earth_r&color_formula=gamma r 2
https://l40l1vwmvi.execute-api.us-west-2.amazonaws.com/cog/tiles/{z}/{x}/{y}.png?url=s3://ornl-cumulus-prod-protected/gedi/GEDI_L4B_Gridded_Biomass/data/GEDI04_B_MW019MW138_02_002_05_R01000M_MU.tif&rescale=0,400&colormap_name=gist_earth_r&color_formula=gamma r 2
https://l40l1vwmvi.execute-api.us-west-2.amazonaws.com/cog/tiles/{z}/{x}/{y}.png?url=s3://nasa-maap-data-store/file-staging/nasa-map/SRTMGL1_COD___001/S10E023.SRTMGL1.tif&rescale=0,400&colormap_name=gist_earth_r&color_formula=gamma r 2
